# Spanish Cadastral data ETL

An example notebook on how I process cadastral info into relevant data for walknet project

In [1]:
repos = r"C:\Users\ManuBenito\Documents\GitHub"
import sys
import os
sys.path.append(repos)

In [2]:
from walknet.sources.spain.landuse.catastro.modules.gatherer import *
from walknet.sources.spain.landuse.catastro.modules.translator import *
from walknet.sources.spain.landuse.catastro.modules.homogenizer import *

**1- Data Gathering**

- Build urls for the INSPIRE serivce, and download spatial data. We will stick to .geojson formats and address data in this project
- Unfortunately, alphanumeric information associated to each municipality needs to be downloaded by hand
- You should end up having a couple folders, one with spatial data and another with the .cat files you downloaded here

In [3]:
PATH_LEVEL0 = r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\land\spain\catastro\level0"

In [4]:
#Madrid Metro Area
#codes = ["28005", "28006", "28007", "28022", "28026", "28045", "28049", "28058", "28065", "28073", "28074", "28080", "28084", "28092", "28104", "28106", "28113", "28115", "28127", "28130", "28134", "28148", "28167", "28176", "28177", "28181", "28184", "28900"]
codes = ["28005"]

In [5]:
download_cadastral_data(codes, PATH_LEVEL0+r"\spatial")

Municipality 28005-ALCALA DE HENARES was successfully downloaded and processed


**2- Data Translation:**

- Process .CAT Files to obtain readable column names and data formats based on [this description](http://www.catastro.minhap.es/documentos/formatos_intercambio/catastro_fin_cat_2006.pdf)
- Merge them with INSPIRE data. I will only use Addresses Points, but parcels and buildings could be also used
- Map Land Use Codes and typology in order to better read and manage them

In [6]:
gdfs = {code: process_cadastral_data(PATH_LEVEL0, code) for code in codes}

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\ManuBenito\\Documents\\Walknet-DataLake\\sources\\land\\spain\\catastro\\level0\\non-spatial'

**2- Data Homogenization:**

- Transform cadastral data into walknet REFERENCE points. This is, geo-points with embedded land use information normalized into walknet land use categories
- In this source, maps combinations of different categorical levels of Land Use and Typology, in order to obtain different use units in a certain point of space

In [ ]:
final = gdfs['28005'][['gml_id',
     'Parcel Cadastral ID',
     'Parcel Province Code','Parcel Province Name','Parcel Municipality Code Cadastre', 'Parcel Municipality Code INE','Parcel Municipality Name INE', 'Parcel Street Code', 'Parcel Street Class', 'Parcel Street Name','Parcel First Number', 'Parcel First Letter', 'Parcel Second Number','Parcel Second Letter',
     'Parcel Area', 'Parcel Built Area', 'Parcel Built Area Above Ground','Parcel Built Area Under Ground', 'Parcel Built Area Under Cover',
     'Building Code ID',
     'Building Year Built', 'Building Accuracy Year Built',
     'Property gml Number', 'Property gml_id',
     'Property ID','Property Cadastral ID',
     'Property First Control Character', 'Property Second Control Character','Property Number Cadastre', 'Property Number Municipality','Property Number Land Register',
     'Property Street Code', 'Property Street Class', 'Property Street Name', 'Property First Number', 'Property First Letter', 'Property Second Number', 'Property Second Letter','Property Kilometric Point',
     'Property Block', 'Property Stairway','Property Floor', 'Property Door Number', 'Property Order Number',
     'Property Age', 'Property General Use','Property Land Use Level1',
     'Property Built Area','Property Other Area (unbuilt or undivided)','Property Coefficient of property if common elements',
     'Property Single or Multiple',
     'Part Order Number',
     'Part Block', 'Part Stairway', 'Part Floor', 'Part Door',
     'Part Detailed Use', 'Part Land Use Level1', 'Part Land Use Level2',
     'Part Renovation Kind', 'Part Renovation Year', 'Part Weighted Year',
     'Part Is Interior', 'Part Is Common',
     'Part Area in Cadastral Terms',
     'Part Area in Terrace or Porch', 'Part Area in Other Floors',
     'Part Typology', 'Part Typology Level1','Part Typology Level2', 'Part Typology Level3','geometry']]

In [ ]:
final.columns

In [ ]:
#We have built the corresponding category combinations in an auxiliary table
relations = pd.read_excel(r"C:\Users\ManuBenito\Documents\GitHub\walknet\sources\spain\landuse\catastro\metadata\relations.xls")

In [ ]:
def apply_logic(df,combination,end_type,category):
    combination.replace(np.nan,['no'])
    df.loc[(df['Property Land Use Level1'].isin(combination_tuple[0]))&
           (df['Part Typology Level1'].isin(combination_tuple[1]))&
           (df['Part Typology Level2'].isin(combination_tuple[2]))&
           (df['Part Typology Level3'].isin(combination_tuple[3]))&
           (df['Part Land Use Level1'].isin(combination_tuple[4]))&
           (df['Part Land Use Level2'].isin(combination_tuple[5])),'Walknet Category'] = end_type+" - "+category
    return df

In [ ]:
df = final

In [ ]:
classes = ['Property Land Use Level1', 'Part Typology Level1', 'Part Typology Level2', 'Part Typology Level3', 'Part Land Use Level1', 'Part Land Use Level2']
for column in classes:
    relations.loc[~relations[column].isna(),column] = relations[column].str.split(";")

In [ ]:
relations

In [ ]:
for i,row in relations.iterrows():
    combination = [row[col] for col in classes]
    end_type = row['End Type']
    category = row['Walknet Category']
    apply_logic(df,combination,end_type,category)

In [ ]:
relations

In [ ]:
for end_type in ["Origins","Destinations"]:
    for category,combination in categories[end_type].items():
        if isinstance(combination,list):
            for case in combination:
                apply_logic(gdf,case,end_type,category)
        else:
            print(combination)
            apply_logic(gdf,combination,end_type,category)

In [ ]:
final['Property Land Use Level1'].unique()

In [ ]:
gdf.loc[~gdf['Part Detailed Use'].isin(["EMS","EBL","ECL","EBS","EIN","EUN","EPR","YGR"]),"Part Detailed Use"] = gdf["Detailed Use"].str[0]

In [ ]:
#Proximal Land Use Categories: (GU,T1,T2,T3,DU)
categories = {
    
"Origins":{
    
"Housing - Single Family Residence":
    (["V","K","A"],["Housing"],["Urban Single Family Residence"],["Isolated or Twin","Detached or Perimeter Block"],["A","V"]),
"Housing - Collective Housing" :
    [(["V"],["Housing"],["Urban Collective Housing"],["Perimeter Block","Open Block"],["V"]),
                       (["O"],["Offices"],["Mixed Use"],["With Housing"],["V"])]
},
"Destinations": {
    
"Retail - Stand Alone Retail":
    [(["C","A","I","O","P"] ,["Retail"], ["Mixed Buildings"],["Store or Workshop"],["C"]),
     (["C"],["Retail"], ["Single Retail Building"],["Only Ground Floor","More Than One Floor"],["C"])],
"Retail - Mall":
    (["C","A","I","O","P"], ["Retail"], ["Mixed Buildings"],["Mall"],["C"]),
"Retail - Market or Supermarket":
    (["C","A","I","O","P"], ["Retail"], ["Market or Supermarket"],["Market","Supermarket"],["C"]),
"Sport - Multiple" :
    (["K","A","I","O","P"], ["Sport"],["Covered","Uncovered"],["Multiple Sports","Swimming Pool"],["K"]),
"Leisure - Sport" :
    (["K","A","I","O","P"], ["Sport"],["Sports - Leisure"], ["Stadiums and Bull Rings","Racecourse, Canodrome and Other"],["K"]),
"Leisure - Shows" :
    (["T","A","O"],["Shows"],["Theatre, Cinema"],["Covered","Uncovered","Theatre","Cinema"],["T","C","G"]),
"Leisure - Bars and Restaurants" :
    [(["T","G"],["Shows"],["Musical Bars, Dancehalls"],["Exclusive Building","With Other Uses"],["G","T","C"]),
     (["G","C","O","T","V"],["Hotels and Restaurants"],["No residence"],["Restaurants","Bars and Coffee Shops"],["C","G","T"])],
"Leisure - Cultural" :
    [(["G","C","O","T","V"],["Hotels and Restaurants"],["Exhibition and Clubs"],["Casinos and Clubs","Exhibition and Congress Halls"],["C","G","T"]),
     (["E","K","A","P"],["Culture and Religion"],["Cultural Without Residence"],["Library and Museum"],["EMS","EBL","ECL"])],
"Healthcare - Multiple" :
    (["Y","P","O","V","C","G"],["Healthcare"],["Hosts Patients","Other","Social Care"],["Clinic, Sanatorium","Hospitals","Primary Health Center","Asylum, Senior Center"],["Y"]),
"Educational - Basic":
    [(["E","K","A","P","V"],["Culture and Religion"],["Cultural Without Residence"],["Faculty or School"],["EBS","EIN"]),
     (["E","K","A","P","V","Y"],["Healthcare"],["Other"],["Kidergarten, Dining Hall"],["YGR"])],
"Educational - Superior":
    (["E","K","A","P"],["Culture and Religion"],["Cultural Without Residence"],["Faculty or School"],["EUN","EPR"]),
"Public - Multiple" :
    (["P","O","Y"],["Heritage, Monumental, Official, Military"],["Official"],["Administrative"],["P","O"])
}
}
    


In [ ]:
#Proximal Land Use Categories: (GU,T1,T2,T3,DU)
categories = {
    
"Origins":{
    
"Housing - Single Family Residence":
    (["V","K","A"],["Housing"],["Urban Single Family Residence"],["Isolated or Twin","Detached or Perimeter Block"],["A","V"]),
"Housing - Collective Housing" :
    [(["V"],["Housing"],["Urban Collective Housing"],["Perimeter Block","Open Block"],["V"]),
                       (["O"],["Offices"],["Mixed Use"],["With Housing"],["V"])]
},
"Destinations": {
    
"Retail - Stand Alone Retail":
    [(["C","A","I","O","P"] ,["Retail"], ["Mixed Buildings"],["Store or Workshop"],["C"]),
     (["C"],["Retail"], ["Single Retail Building"],["Only Ground Floor","More Than One Floor"],["C"])],
"Retail - Mall":
    (["C","A","I","O","P"], ["Retail"], ["Mixed Buildings"],["Mall"],["C"]),
"Retail - Market or Supermarket":
    (["C","A","I","O","P"], ["Retail"], ["Market or Supermarket"],["Market","Supermarket"],["C"]),
"Sport - Multiple" :
    (["K","A","I","O","P"], ["Sport"],["Covered","Uncovered"],["Multiple Sports","Swimming Pool"],["K"]),
"Leisure - Sport" :
    (["K","A","I","O","P"], ["Sport"],["Sports - Leisure"], ["Stadiums and Bull Rings","Racecourse, Canodrome and Other"],["K"]),
"Leisure - Shows" :
    (["T","A","O"],["Shows"],["Theatre, Cinema"],["Covered","Uncovered","Theatre","Cinema"],["T","C","G"]),
"Leisure - Bars and Restaurants" :
    [(["T","G"],["Shows"],["Musical Bars, Dancehalls"],["Exclusive Building","With Other Uses"],["G","T","C"]),
     (["G","C","O","T","V"],["Hotels and Restaurants"],["No residence"],["Restaurants","Bars and Coffee Shops"],["C","G","T"])],
"Leisure - Cultural" :
    [(["G","C","O","T","V"],["Hotels and Restaurants"],["Exhibition and Clubs"],["Casinos and Clubs","Exhibition and Congress Halls"],["C","G","T"]),
     (["E","K","A","P"],["Culture and Religion"],["Cultural Without Residence"],["Library and Museum"],["EMS","EBL","ECL"])],
"Healthcare - Multiple" :
    (["Y","P","O","V","C","G"],["Healthcare"],["Hosts Patients","Other","Social Care"],["Clinic, Sanatorium","Hospitals","Primary Health Center","Asylum, Senior Center"],["Y"]),
"Educational - Basic":
    [(["E","K","A","P","V"],["Culture and Religion"],["Cultural Without Residence"],["Faculty or School"],["EBS","EIN"]),
     (["E","K","A","P","V","Y"],["Healthcare"],["Other"],["Kidergarten, Dining Hall"],["YGR"])],
"Educational - Superior":
    (["E","K","A","P"],["Culture and Religion"],["Cultural Without Residence"],["Faculty or School"],["EUN","EPR"]),
"Public - Multiple" :
    (["P","O","Y"],["Heritage, Monumental, Official, Military"],["Official"],["Administrative"],["P","O"])
}
}
    




In [ ]:
def apply_logic(df,combination_tuple,end_type,category):
    df.loc[(df['Property General Use'].isin(combination_tuple[0]))&
           (df['Part Tipology Level1'].isin(combination_tuple[1]))&
           (df['Tipology Level2'].isin(combination_tuple[2]))&
           (df['Tipology Level3'].isin(combination_tuple[3]))&
           (df['Detailed Use'].isin(combination_tuple[4])),'Proximity Category'] = end_type+" - "+category
    return df

In [ ]:
for end_type in ["Origins","Destinations"]:
    for category,combination in categories[end_type].items():
        if isinstance(combination,list):
            for case in combination:
                apply_logic(gdf,case,end_type,category)
        else:
            print(combination)
            apply_logic(gdf,combination,end_type,category)

In [ ]:
gdf

In [ ]:
common = ['Province Code', 'Province Name', 'Municipality Code Cadastre','Municipality Code INE', 'Municipality Name INE', 'Settlement','Postal Code', 'Municipality District','Original Municipality Code','Zone Code', 'Polygon Code', 'Cadastral Parcel ID',    
'Parcel Area', 'Built Area','Built Area Above Ground', 'Built Area Under Ground',
'Built Area Under Cover',  'Special Asset', 'Special Asset Name','Building Class (Urban, Rural, Special)', 'Building Code ID',
'Year Built', 'Accuracy Year Built', 'Facade Lenght', 'Matrix Building','gml_id', 'localId', 'namespace', 'specification',
'address_id','Cadastral Property ID','Property ID', 'Street Code Property','Street Class Property', 'Street Name Property','First Number Property', 'First Letter Property',
'Proximity Category']

#'Age of Property', 'General Use', 'Built Area Property','Single or Multiple Property','Detailed Use', 'Renovation Kind','Renovation Year', 'Weighted Year', 'Interior' ,'Tipology','Is Common','Tipology Level1', 'Tipology Level2', 'Tipology Level3']
local = ['Area in Cadastral Terms']

In [ ]:
t = gdf.groupby(common).sum()[local]

In [ ]:
t

In [ ]:
t.head(25000).to_excel(path+r"\test.xlsx")

In [ ]:
t = gdf.groupby(['Cadastral Parcel ID','Street Class Property','Street Name Property','First Number Property','Tipology Level1','Tipology Level2','Tipology Level3'])['Area in Cadastral Terms'].sum()

In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
t

In [ ]:
#t = t.pivot(index = ['Cadastral Parcel ID','Street Class Property','Street Name Property','First Number Property'],columns=['Tipology Level1','Tipology Level2','Tipology Level3'],values='Area in Cadastral Terms').fillna(0)

In [ ]:
t

In [ ]:
t.to_excel(path+r"\test.xlsx")

In [ ]:
t = gdf.groupby(['Cadastral Parcel ID','Property ID','General Use','Tipology'])['Area in Cadastral Terms'].sum().reset_index()

In [ ]:
gdf['Tipology'].unique()

In [ ]:
#t.to_csv(path+r"\teste.csv",sep=";")
t.groupby(['Cadastral Parcel ID','Property ID','Tipology'])['Area in Cadastral Terms'].count()

In [ ]:
codes_columns = list(t.columns)
totals = t[codes_columns].sum(axis=1)
for c in t.columns:
    t["p"+c] = (100*t[c])/totals

In [ ]:
t['sort_use'] = t.apply(list, 1).apply(sorted).apply(list)

In [ ]:
t['sort_use'] = t.apply(lambda r:[el for el in r['sort_use'] if el !=0.0],axis=1)

In [ ]:
t['sort_use']

In [ ]:
#gdf.to_file(path+r"\test.gpkg",driver='GPKG')

In [ ]:
df11 = prepare_df(leecat11(cadastre_path))
df13 = prepare_df(leecat13(cadastre_path))
df14 = prepare_df(leecat14(cadastre_path))
df15 = prepare_df(leecat15(cadastre_path))

In [ ]:
cityname = df11['NOMMUN'].iloc[0]
print("Reconocido municipio: "+cityname)

In [ ]:
df11 = df11[['PCAT','CODPROV','CODINE','NOMPROV','NOMMUN','PCODVIA','PTIPVIA','PNOMVIA','PPNP','PPLE','SUPP','SUPCT','SUPCSR','SUPCBR','SUPCUB','COORX','COORY','DEN_BICE']]
df13 = df13[['PCAT','COD_UC','ANOCONS','ANOEXAC']]
df14 = df14[['PCAT','NCBI_PC','COD_UC','CDD','TIPREF','ANOREF','ANOEFE','SUPT_CAT']]
#Drop common spaces
df14 = df14.loc[df14['NCBI_PC'] != ""]
df15 = df15[['PCAT','NCBI_PC','CODVIA','TIPVIA','NOMVIA','PNP','PLE','NOBI','ANOANT','CDD_BI','SUPCON']]

In [ ]:
#Merge all four register tables
df = merge_and_drop(df15,df14,'right',['PCAT','NCBI_PC'])
df = merge_and_drop(df13,df,'right',['PCAT','COD_UC'])
df = merge_and_drop(df11,df,'right',['PCAT'])
df['PPNP'] = df['PPNP'].fillna(0).astype('int64').astype('str').str.replace("0", "")
df['PNP'] = df['PNP'].fillna(0).astype('int64').astype('str').str.replace("0", "")

In [ ]:
len(df.PCAT.unique())

In [ ]:
df_bi = df.drop_duplicates(subset = ['PCAT','NCBI_PC'],keep='first')

In [ ]:
sums = pd.pivot_table(df_bi, values='SUPCON', index=['PCAT'],columns=['CDD_BI'], aggfunc=np.sum).fillna(0)

In [ ]:
counts = pd.pivot_table(df_bi, values='SUPCON', index=['PCAT'],columns=['CDD_BI'], aggfunc=lambda x: len(x.unique())).fillna(0)

In [ ]:
sums.rename(columns = {c:"S"+c for c in sums.columns},inplace=True)

In [ ]:
counts.rename(columns = {c:"N"+c for c in counts.columns},inplace=True)

In [ ]:
addresses_inspire = gpd.read_file(addresses_inspire)

In [ ]:
#addresses_inspire

In [ ]:
parcels_loaded = pd.merge(parcels,counts,left_index=True,right_index=True,how='left')
parcels_loaded = pd.merge(parcels,sums,left_index=True,right_index=True,how='left')

In [ ]:
for p in [c for c in parcels_loaded.columns if len(c)==2]:
    parcels_loaded[p].fillna(0,inplace=True)

In [ ]:
len(parcels_loaded)

In [ ]:
parcels_loaded.to_file(path+cityname+"_loaded.shp",driver='ESRI Shapefile')

# Get Land Use Reference points ready
Basically, I use cadastral addresses points described [here](http://www.catastro.minhap.es/ayuda/manual_descriptivo_shapefile.pdf), assign the nearest parcel id to each one,and then try to match them to my cadastral records. This point is the closest I can get to the actual entrance of each land use activity, I call it ***'refpoint'***

In [ ]:
addresses = gpd.read_file(access_file)
#addresses = addresses.loc[addresses['INTEREST']==1]
addresses = addresses.loc[~addresses['geometry'].isna()]
parcels = gpd.read_file(parcels_file)
#parcels = parcels.loc[parcels['INTEREST']==1]
parcels = parcels.loc[~parcels['geometry'].isna()]

In [ ]:
#Get the address points
addresses = addresses.loc[addresses['TTGGSS'] == '189401']
addresses['geometry'] = addresses['geometry'].centroid

In [ ]:
addresses.head(2)

In [ ]:
parcels.head(2)

In [ ]:
fig, ax = plt.subplots(figsize =(10,10))
ax.set_facecolor('black')
parcels.plot(ax=ax)
addresses.plot(ax=ax,
                    color='lightgray',
                    markersize = 2)
plt.grid(True,color='w', linestyle='-', linewidth=0.3)
plt.show();
plt.clf()

In [ ]:
%run Utils.ipynb

#Find Nearest Parcels
addresses = find_nearest(addresses,parcels,'REFCAT')[['ROTULO','REFCAT','geometry']]
display(addresses.head())

In [ ]:
#Three alternatives (Proper Address, Parcel Address, Parcel Centroid)
addresses['gaddress'] = addresses['REFCAT']+addresses['ROTULO']
addresses = addresses[['REFCAT','ROTULO','gaddress','geometry']]
df['address'] = df['PCAT']+df['PNP']+df['PLE']
df['paddress'] = df['PCAT']+df['PPNP']+df['PPLE']
dict_address_points = pd.Series(addresses.geometry.values,index=addresses.gaddress).to_dict()
df['refpoint'] = df['address'].map(dict_address_points)
dfa = df[df['refpoint'].notnull()]
dfb = df[df['refpoint'].isnull()]
dfb['refpoint'] = dfb['paddress'].map(dict_address_points)
dfc = dfb[dfb['refpoint'].isnull()]             
dfb = dfb[dfb['refpoint'].notnull()]
dfc['refpoint'] = dfc.apply(lambda row: Point(row.COORX, row.COORY), axis=1) 
reference_points = pd.concat([dfa,dfb,dfc])
reference_points['geometry'] = reference_points['refpoint'] 
display(reference_points.head())

# Process network data and find possible network reference points for land use

Now I need another point where my addresses (entrances) snap to the network, in order to create a significant point of activity along my street network edges. I create a set of possible measure points and snap my addresses to it

In [ ]:
network = gpd.read_file(network_file)
#network = network.loc[network['INTEREST2']==1]

In [ ]:
#Create measure points by splitting original network into 50 m chunks and retrieving all the resulting line intersections
spl = lines_and_interpolated_vertices(network,100,['id_vial','id_tramo','tipo_vial','clase','firme','tipovehic'])
split_interpolated_network = spl[0]
modified_network = spl[1]

In [ ]:
measure_points = get_lines_endpoints(split_interpolated_network)
measure_points = gpd.GeoDataFrame(measure_points,columns=['geometry'])
measure_points['netpoint'] = measure_points.index
display(measure_points.head())

In [ ]:
print(len(reference_points))

In [ ]:
#Find Nearest Parcels
final_registers = find_nearest(reference_points,measure_points,'netpoint')
final_registers = pd.merge(final_registers,measure_points,how='left',on='netpoint')
final_registers['netpointg'] = final_registers['geometry_y']
final_registers.drop(columns=['geometry_x','geometry_y'],inplace=True)

# Prepare final network geometry

Now I want to create edges geometries that are aware of all intersections in my original data, as well as the identified network reference points, so I optimize the number of edges used

First, I will grab all the intersections of the original data

In [ ]:
network['geometry'] = network['geometry'].astype('str')
original_endpoints = [p.wkt for p in get_lines_endpoints(network)]
points_with_activity =  [p.wkt for p in list(final_registers.netpointg.unique())]
final_network_points = list(set(original_endpoints+points_with_activity))

Then, I will create a network that is split on the new interpolated points

In [ ]:
final_split_network = split_lines_at_points(modified_network,final_network_points,['id_vial','id_tramo','tipo_vial','clase','firme','tipovehic'],'id_tramo')

In [ ]:
final_split_network = calculate_speed_by_slope(final_split_network,'length','slope')

Finally, I need to referr my cadastral registers to the final network points, as the set has changed significantly. I will recreate the index column and relate my reference points to these again

In [ ]:
final_network_points = pd.DataFrame(final_network_points)
final_network_points['netpoint'] = final_network_points.index

In [ ]:
final_registers.drop(columns=['netpoint','netpointg'],inplace=True)

In [ ]:
display(final_registers.head())

In [ ]:
final_registers['geometry'] = final_registers['refpoint']
final_network_points.rename(columns = {0:'geometry'},inplace = True)

In [ ]:
final_network_points['geometry'] = final_network_points.apply(lambda row: Point(loads(row['geometry'])),axis=1)

In [ ]:
final_network_points.to_csv(path+"\\MADRID\\final_network_points.csv")

In [ ]:
final_network_points = gpd.GeoDataFrame(final_network_points,geometry ='geometry')

In [ ]:
final_registers = find_nearest(final_registers,final_network_points,'netpoint')

In [ ]:
final_registers.rename(columns = {'geometry':'netpointg'},inplace=True) 

In [ ]:
final_registers.to_csv(path+"\\MADRID\\CAT_referenciado.csv")
final_network_points.to_csv(path+"\\MADRID\\RED_nodos.csv")

In [ ]:
measure_points.to_csv(path+"\\MADRID\\measure_points.csv")

In [ ]:
final_split_network['geometry'] = final_split_network['geometry'].apply(loads)

final_split_network = gpd.GeoDataFrame(final_split_network,geometry='geometry')
final_split_network.to_file(path+"\\MADRID\\edges_loaded.shp",driver = 'Shapefile')

In [ ]:
final_split_network.to_csv(path+"\\MADRID\\edges_loaded.csv")